In [ ]:
# ====================================================
# Library
# ====================================================
import os
import re
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
import torch

# ====================================================
# Preprocess functions
# ====================================================
def split_form(form):
    string = ''
    for i in re.findall(r"[A-Z][^A-Z]*", form):
        elem = re.match(r"\D+", i).group()
        num = i.replace(elem, "")
        if num == "":
            string += f"{elem} "
        else:
            string += f"{elem} {str(num)} "
    return string.rstrip(' ')

def split_form2(form):
    string = ''
    for i in re.findall(r"[a-z][^a-z]*", form):
        elem = i[0]
        num = i.replace(elem, "").replace('/', "")
        num_string = ''
        for j in re.findall(r"[0-9]+[^0-9]*", num):
            num_list = list(re.findall(r'\d+', j))
            assert len(num_list) == 1, f"len(num_list) != 1"
            _num = num_list[0]
            if j == _num:
                num_string += f"{_num} "
            else:
                extra = j.replace(_num, "")
                num_string += f"{_num} {' '.join(list(extra))} "
        string += f"/{elem} {num_string}"
    return string.rstrip(' ')

import re
PATTEN = re.compile('\d+|[A-Z][a-z]?|[^A-Za-z\d/]|/[a-z]')
def l_split(s):
    return ' '.join(re.findall(PATTEN,s))

# ====================================================
# Tokenizer
# ====================================================
class Tokenizer(object):
    
    def __init__(self):
        self.stoi = {}
        self.itos = {}

    def __len__(self):
        return len(self.stoi)
    
    def fit_on_texts(self): 
        vocab = [str(i) for i in range(180)]
        vocab.extend(['(', ')', '+', ',', '-', '/b', '/c', '/h', '/i', '/m', '/s', '/t'
                         , 'B', 'Br', 'C', 'Cl', 'F', 'H', 'I', 'N', 'O', 'P', 'S', 'Si', 'T', 'D'
                         , '<sos>', '<eos>', '<pad>'])
#         ['B', 'Br', 'C', 'Cl', 'F', 'H', 'I', 'N', 'O', 'P', 'S', 'Si']
#         {'b', 'm', 'i', 'c', 't', 's', 'h'}
        for i, s in enumerate(vocab):
            self.stoi[s] = i
        self.itos = {item[1]: item[0] for item in self.stoi.items()}

    
#     def fit_on_texts(self, texts):
#         vocab = set()
#         for text in texts:
#             vocab.update(text.split(' '))
#         vocab = sorted(vocab)
#         vocab.append('<sos>')
#         vocab.append('<eos>')
#         vocab.append('<pad>')
#         for i, s in enumerate(vocab):
#             self.stoi[s] = i
#         self.itos = {item[1]: item[0] for item in self.stoi.items()}
        
    def text_to_sequence(self, text):
        sequence = []
        sequence.append(self.stoi['<sos>'])
        for s in text.split(' '):
            sequence.append(self.stoi[s])
        sequence.append(self.stoi['<eos>'])
        return sequence
    
    def texts_to_sequences(self, texts):
        sequences = []
        for text in texts:
            sequence = self.text_to_sequence(text)
            sequences.append(sequence)
        return sequences

    def sequence_to_text(self, sequence):
        return ''.join(list(map(lambda i: self.itos[i], sequence)))
    
    def sequences_to_texts(self, sequences):
        texts = []
        for sequence in sequences:
            text = self.sequence_to_text(sequence)
            texts.append(text)
        return texts
    
    def predict_caption(self, sequence):
        caption = ''
        for i in sequence:
            if i == self.stoi['<eos>'] or i == self.stoi['<pad>']:
                break
            caption += self.itos[i]
        return caption
    
    def predict_captions(self, sequences):
        captions = []
        for sequence in sequences:
            caption = self.predict_caption(sequence)
            captions.append(caption)
        return captions

In [ ]:
# ====================================================
# Data Loading
# ====================================================
train = pd.read_csv('../input/bms-molecular-translation/train_labels.csv')
print(f'train.shape: {train.shape}')

In [ ]:
# ====================================================
# main
# ====================================================
# def main():
# ====================================================
# preprocess train.csv
# ====================================================
train['InChI_text'] = train['InChI'].progress_apply(lambda x: l_split(x[9:]))
# ====================================================
# create tokenizer
# ====================================================
tokenizer = Tokenizer()
tokenizer.fit_on_texts()
#     train['InChI_text'].values
print(f"tokenizer.stoi: {tokenizer.stoi.keys()}")
torch.save(tokenizer, 'tokenizer2.pth')
print('Saved tokenizer')
# ====================================================
# preprocess train.csv
# ====================================================
lengths = []
tk0 = tqdm(train['InChI_text'].values, total=len(train))
for text in tk0:
    seq = tokenizer.text_to_sequence(text)
    length = len(seq) - 2
    lengths.append(length)
train['InChI_length'] = lengths
train.to_pickle('train2.pkl')
print('Saved preprocessed train.pkl')
print(train['InChI_text'].iloc[2])

In [ ]:
wrong = []
for i in tqdm(range(len(train))):
    new = l_split(train.InChI[i][9:])
    if train['InChI_text'][i].replace(" ", "") != train.InChI[i][9:]:
        wrong.append(i)
print(len(wrong))